# Featre Engineering

### Imports & Data Load

In [97]:
%pip install --quiet pandas numpy
import pandas as pd
import numpy as np
import os

Note: you may need to restart the kernel to use updated packages.


In [98]:
dataset_path = 'data/raw/visuelle2'

# Load your datasets (update the file paths as needed)
customer_df = pd.read_csv(os.path.join(dataset_path, 'customer_data.csv'))
discounts_df = pd.read_csv(os.path.join(dataset_path, 'price_discount_series.csv'))
restocks_df = pd.read_csv(os.path.join(dataset_path, 'restocks.csv'))
sales_df = pd.read_csv(os.path.join(dataset_path, 'sales.csv'))
trends_df = pd.read_csv(os.path.join(dataset_path, 'vis2_gtrends_data.csv'))
weather_df = pd.read_csv(os.path.join(dataset_path, 'vis2_weather_data.csv'))


### Trend Dataset Feature Engineering

Objective: Aggregate Google Trends by week and category.

In [95]:
# Convert date column to datetime and get week number
trends_df['date'] = pd.to_datetime(trends_df['date'])
trends_df['year'] = trends_df['date'].dt.isocalendar().year
trends_df['week'] = trends_df['date'].dt.isocalendar().week

# Melt so each row = [date, category, value]
trend_long = trends_df.melt(id_vars=['date', 'year', 'week'], var_name='category', value_name='trend')

# Now group by week and category
weekly_trends = trend_long.groupby(['date', 'year', 'week', 'category'], as_index=False)['trend'].mean()

weekly_trends.to_parquet('./data/processed/weekly_trends.parquet')
weekly_trends

,date,year,week,category,trend
0,2015-10-05,2015,41,acrylic,57.0
1,2015-10-05,2015,41,angora,65.0
2,2015-10-05,2015,41,bengaline,13.0
3,2015-10-05,2015,41,black,25.0
4,2015-10-05,2015,41,blue,77.0
...,...,...,...,...,...
21115,2019-12-16,2019,51,violet,59.0
21116,2019-12-16,2019,51,viscose twill,80.0
21117,2019-12-16,2019,51,webbing,48.0
21118,2019-12-16,2019,51,white,92.0


### Weather Dataset Feature Engineering

Objective: Aggregate weather by week and locality.

In [100]:
import pandas as pd
import datetime

# Ensure 'date' is datetime with day-first format
weather_df['date'] = pd.to_datetime(weather_df['date'], dayfirst=True)
# Extract ISO year and week
weather_df['year'] = weather_df['date'].dt.isocalendar().year
weather_df['week'] = weather_df['date'].dt.isocalendar().week

# Aggregate daily weather data to weekly level per locality (do not average locality!)
weather_weekly = weather_df.groupby(['locality', 'year', 'week'], as_index=False).mean(numeric_only=True)

# Compute the actual Monday (first day) for each ISO week (ISO 8601 standard)
def iso_week_start(row):
    return datetime.date.fromisocalendar(int(row['year']), int(row['week']), 1)
weather_weekly['date'] = weather_weekly.apply(iso_week_start, axis=1)
weather_weekly['date'] = pd.to_datetime(weather_weekly['date'])  # Convert to pandas.Timestamp

# Reorder columns to have 'date' at the front
cols = ['date', 'year', 'week', 'locality'] + [col for col in weather_weekly.columns if col not in ['date', 'year', 'week', 'locality']]
weather_weekly = weather_weekly[cols]

# Drop the first row (commonly because it refers to an incomplete week at the dataset boundary)
weather_weekly = weather_weekly.iloc[1:].reset_index(drop=True)

# Save aggregated weekly weather to parquet file
weather_weekly.to_parquet('./data/processed/weekly_weather_agg.parquet', index=False)
weather_weekly.head(10)

,date,year,week,locality,avg temp °C,min temp °C,max temp °C,dew point °C,humidity %,visibility km,avg wind km/h,max wind km/h,gust km/h,slm pressure mb,avg pressure mb,rain mm
0,2016-01-04,2016,1,8,13.142857,8.857143,17.142857,9.000000,76.714286,11.285714,11.857143,23.285714,19.000000,1005.000000,0.0,0.0
1,2016-01-11,2016,2,8,11.285714,7.142857,15.428571,5.000000,63.285714,12.000000,17.000000,29.571429,34.571429,1009.571429,0.0,0.0
2,2016-01-18,2016,3,8,6.714286,3.142857,9.857143,2.571429,69.000000,9.571429,13.714286,20.428571,7.428571,1023.000000,0.0,0.0
3,2016-01-25,2016,4,8,10.285714,5.571429,15.285714,6.571429,77.142857,8.285714,9.571429,17.428571,0.000000,1028.571429,0.0,0.0
4,2016-02-01,2016,5,8,11.000000,6.428571,15.285714,6.285714,71.571429,11.857143,13.857143,22.857143,18.000000,1022.285714,0.0,0.0
5,2016-02-08,2016,6,8,13.000000,8.000000,17.714286,8.000000,72.142857,13.428571,12.857143,25.000000,26.000000,1008.142857,0.0,0.0
6,2016-02-15,2016,7,8,14.000000,9.857143,18.142857,9.285714,74.142857,11.571429,11.857143,22.857143,25.857143,1016.285714,0.0,0.0
7,2016-02-22,2016,8,8,13.285714,8.428571,17.571429,8.857143,74.714286,11.142857,12.571429,22.571429,17.714286,1012.285714,0.0,0.0
8,2016-02-29,2016,9,8,11.714286,7.428571,15.857143,6.428571,69.857143,11.714286,14.714286,31.142857,42.285714,1005.714286,0.0,0.0
9,2016-03-07,2016,10,8,10.857143,6.285714,14.857143,7.142857,77.714286,10.571429,11.857143,21.428571,9.857143,1008.142857,0.0,0.0


### Sales Dataset Feature Engineering

Objective: Aggregate sales by external_code, week, and retail.

In [82]:
# List of columns representing weeks after release
sales_columns = [str(i) for i in range(12)]

# Unpivot: transform the data into a long table, with one row per week after the release
sales_long = sales_df.melt(
    id_vars=['external_code', 'retail', 'release_date', 'season', 'category', 'color', 'image_path', 'fabric'],  # ID columns
    value_vars=sales_columns,  # Week columns to be unpivoted
    var_name='week_offset',  # New column name for the week offset
    value_name='qty_sold'  # New column name for quantity sold
)

# Calculate the actual sale date based on release date and week offset
sales_long['week_offset'] = sales_long['week_offset'].astype(int)  # Ensure week_offset is an integer
# Ensure release_date is datetime
sales_long['release_date'] = pd.to_datetime(sales_long['release_date'], dayfirst=True)

# Calculate effective sale date
sales_long['effective_sale_date'] = sales_long['release_date'] + pd.to_timedelta(sales_long['week_offset'] * 7, unit='D')

# Extract year and ISO week number from the effective sale date
sales_long['year'] = sales_long['effective_sale_date'].dt.isocalendar().year  # Get ISO year
sales_long['week'] = sales_long['effective_sale_date'].dt.isocalendar().week  # Get ISO week number

# Group by keys and sum the sales
weekly_sales = sales_long.groupby(
    ['external_code', 'retail', 'year', 'week', 'release_date', 'season', 'category', 'color', 'image_path', 'fabric'],
    as_index=False
)['qty_sold'].sum()

# Save the result to a Parquet file
weekly_sales.to_parquet('./data/processed/weekly_sales.parquet')
weekly_sales.head(13)


/var/folders/tc/y68ss1d508g5s83sx4mrflf80000gq/T/ipykernel_62421/2501906300.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  sales_long['release_date'] = pd.to_datetime(sales_long['release_date'], dayfirst=True)


,external_code,retail,year,week,release_date,season,category,color,image_path,fabric,qty_sold
0,1,3,2016,49,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,3.0
1,1,3,2016,50,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,3.0
2,1,3,2016,51,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,1.0
3,1,3,2016,52,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,4.0
4,1,3,2017,1,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,3.0
5,1,3,2017,2,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,0.0
6,1,3,2017,3,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,1.0
7,1,3,2017,4,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,0.0
8,1,3,2017,5,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,0.0
9,1,3,2017,6,2016-12-05,SS17,long sleeve,violet,PE17/00001.png,acrylic,0.0


### Restock Dataset Feature Engineering

Objective: Aggregate restock information by external_code, week, and retail.

In [83]:
weekly_restock = restocks_df.groupby(['external_code', 'retail', 'year', 'week'], as_index=False)['qty'].sum()

weekly_restock.to_parquet('./data/processed/weekly_restock.parquet')
weekly_restock

,external_code,retail,year,week,qty
0,1,0,2016,48,4
1,1,1,2016,48,4
2,1,1,2016,51,7
3,1,2,2016,48,4
4,1,2,2016,52,6
...,...,...,...,...,...
949760,5577,117,2020,2,4
949761,5577,121,2019,51,7
949762,5577,125,2019,51,7
949763,5577,125,2020,1,1


### Discounts Dataset Feature Engineering

#### Main operations:
- **Unpivoting**: Converted the wide format discount data (columns "0"–"11", representing weeks since product release) to a long format table, so each row corresponds to a product, retailer, and specific week after release.
- **Date alignment**: For each discount week, the corresponding real-world calendar date was computed using the product's `release_date` from the sales dataset. From this date, we extracted the ISO year and week number.
- **Normalized discounted price**: For each product/week/retail, we calculated the normalized discounted price, which represents the *relative* price after discount, compared to a (confidential) reference value. This allows for cross-product and cross-period comparison without exposing actual price values.

#### Why normalized discounted price?

The **normalized discounted price** is crucial for:
- Enabling price sensitivity analyses and promo effectiveness studies without revealing confidential pricing.
- Consistently comparing how discounts influence product pricing and sales across different products, retailers, and time periods.
- Using price as an input feature for machine learning models, since it is already standardized.

The resulting table is now sorted by `external_code`, `retail`, `year`, and `week`, making it ready for further merging, analysis, and modeling.

In [84]:
import pandas as pd

# Unpivot week columns (0–11) into long format
discounts_long = discounts_df.melt(
    id_vars=['external_code', 'retail', 'price'],
    value_vars=[str(i) for i in range(12)],
    var_name='week_since_release',
    value_name='discount_value'
)
discounts_long['week_since_release'] = discounts_long['week_since_release'].astype(int)

# Get release_date from sales_df for each (external_code, retail) pair
release_dates = sales_df[['external_code', 'retail', 'release_date']].drop_duplicates()
release_dates['release_date'] = pd.to_datetime(release_dates['release_date'])

# Merge discounts_long with release_dates
discounts_long = discounts_long.merge(
    release_dates,
    on=['external_code', 'retail'],
    how='left'
)

# Compute real calendar date for the discount week
discounts_long['effective_discount_date'] = discounts_long['release_date'] + pd.to_timedelta(discounts_long['week_since_release'] * 7, unit='D')
discounts_long['year'] = discounts_long['effective_discount_date'].dt.isocalendar().year
discounts_long['week'] = discounts_long['effective_discount_date'].dt.isocalendar().week

# Compute normalized discounted price for each specific week
discounts_long['normalized_discounted_price'] = discounts_long['price'] * (1 - discounts_long['discount_value'])

# Select and sort columns as required
discounts_long = discounts_long[
    ['external_code', 'retail', 'year', 'week', 'discount_value', 'normalized_discounted_price']
].sort_values(
    by=['external_code', 'retail', 'year', 'week']
).reset_index(drop=True)

# Save to Parquet
discounts_long.to_parquet('./data/processed/weekly_discounts.parquet', index=False)
discounts_long

,external_code,retail,year,week,discount_value,normalized_discounted_price
0,1,3,2016,49,0.17,0.045604
1,1,3,2016,50,0.00,0.054944
2,1,3,2016,51,0.05,0.052197
3,1,3,2016,52,0.00,0.054944
4,1,3,2017,1,0.00,0.054944
...,...,...,...,...,...,...
1282195,5577,108,2020,6,0.50,0.038421
1282196,5577,108,2020,7,0.50,0.038421
1282197,5577,108,2020,8,0.50,0.038421
1282198,5577,108,2020,9,0.72,0.021516


### Customer Dataset Feature Engineering

Objective: Match customer purchases to discount periods/products.

In [85]:
# Rename 'data' to 'date'
customer_df = customer_df.rename(columns={'data': 'date'})

# Ensure 'date' is datetime
customer_df['date'] = pd.to_datetime(customer_df['date'])

# Extract ISO year and week from the date
customer_df['year'] = customer_df['date'].dt.isocalendar().year
customer_df['week'] = customer_df['date'].dt.isocalendar().week

# Optional: Reorder columns
customer_df = customer_df[['customer', 'retail', 'external_code', 'date', 'year', 'week', 'qty']]

# Save as parquet
customer_df.to_parquet('./data/processed/weekly_customers.parquet', index=False)
customer_df

,customer,retail,external_code,date,year,week,qty
0,1,0,940,2017-06-10 12:41:57,2017,23,1
1,1,0,718,2017-06-10 12:41:57,2017,23,1
2,1,0,825,2017-06-10 12:54:11,2017,23,1
3,2,1,471,2017-08-03 11:37:27,2017,31,1
4,2,1,613,2017-08-03 11:37:27,2017,31,1
...,...,...,...,...,...,...,...
3184157,667076,20,5356,2021-03-01 16:49:38,2021,9,1
3184158,667077,20,5356,2021-03-04 19:36:50,2021,9,1
3184159,667078,20,4630,2021-03-10 12:43:44,2021,10,1
3184160,667082,12,4539,2021-04-27 16:02:16,2021,17,1
